# Sentiment Analysis of Twitter Data

### installing required libraries
twint library used for colliction data from twitter without API.

In [1]:
!pip install twint
!pip install --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint
#this prevents async problems/ runtime errors
!pip install nest_asyncio

  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.7/263.7 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.3/385.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.8 MB/s eta 0:00:00
  Created wheel for twint: filename=twint-2.1.20-py3-none-any.whl size=33929 sha256=92bd40f5702aafff48d2153ecd3ae1d0f6f1df1b1f885ce8a5146cafea4753bc
  Stored in directory: /root/.cache/pip/wheels/44/fc/77/99887a36b5c265a87516158858697d1a0b8f32c4d4dbddbb24
  Created wheel for googletransx: filename=googletransx-2.4.2-py3-none-any.whl size=15968 sha256=3dbf065f4e6acd16d5b477c079942e6c13ac939d1817f4aad0f5ebf908978d2b
  Stored 

### import required libraries

In [2]:
import twint
import nest_asyncio
nest_asyncio.apply()
import json
import re
from nltk import ngrams
import csv
import pandas as pd

### Collection data
Instantiate and configure the twint-object for get data from twitter.


In [3]:
c = twint.Config()
c.Store_object = True
c.Pandas =True
c.Search = "#سياحة"
# c.Since = '2022-01-01'
# c.Until = '2022-11-30'
c.Limit = 100
c.Lang = 'ar'
c.Filter_retweets = False
c.Store_json = True
c.Output = "1_tweets.json"
twint.run.Search(c)
# Quick check
# twint.storage.panda.Tweets_df.head(20)

1603509468301381632 2022-12-15 21:57:03 +0000 <ger_capital> السياحة في دولة جورجيا  أفضل العروض بأفضل ألأسعار مع جورجيا كابيتال  واتساب: 00995514205050 تبليسى #جورجيا #جورجيا_تبليسي #سياحة #عطلات #اجازة #سافر #رحلات_طوال_العام  https://t.co/KV8bpUAuVL
1603508209045491712 2022-12-15 21:52:03 +0000 <QalWQalt> معانا هتزور احلا مناطق للسياحه في #جورجيا مع جورجيا كابيتال  #سياحة #سفر  https://t.co/8ZlsAPhzb2
1603507955839643649 2022-12-15 21:51:03 +0000 <QalWQalt> افضل #سياحة عائلية في اذربيجان واتساب: 00994508225050  https://t.co/IrQnE3GqWi
1603502277410533377 2022-12-15 21:28:29 +0000 <safari_news> بالصور .. أفضل 3 أماكن ترفيهية للعوائل في #سان_خوسيه الأميركية   https://t.co/WdSloZWHN6  #سياحة  https://t.co/JMWpgkeJk3
1603499512655388683 2022-12-15 21:17:30 +0000 <amfhd50387603> ننبهر بجمال الدول الغربية وخصوصاً الاوروبية لأننا نزورها كسياح نسكن الفنادق والمنتجعات ونأكل بالمطاعم ونتبضع بالاسواق لكن لمن يعيش ويعمل فيها يشعر بصرامة وقساوة القانون وشدة العقوبة وسطوة الضرائب وإنفلات الاخلاق ف

## 1- Loading and Cleaning Twitter Data

In [4]:
# Open the input file
input_file = open('/kaggle/input/sentiment-arabic-tweets/1_tweets.json','r', encoding="utf-8")
clean_tweets = open('/kaggle/working/clean_tweets.txt','w', encoding="utf-8")


# Load the first few lines
for line in input_file:
    tweet = json.loads(line)
    temp = re.sub("@[A-Za-z0-9_]+","@User", tweet['tweet'])
    temp = re.sub(r"http\S+", "URL", temp)
    temp = re.sub(r"www.\S+", "URL", temp)
    temp = re.sub('[()!\'?]', '', temp)
    temp = re.sub('\[.*?\]','', temp)
    temp = re.sub("[^أ-يA-z0-9@]"," ", temp)
    clean_tweets.write(temp + '\n')

# Close the input file
input_file.close()
clean_tweets.close()

## 2- Derive the sentiment of each tweet

put the lexicon/Afinn file in dict as {term:score}

In [5]:
afinnfile = open("/kaggle/input/sentiment-arabic-tweets/3_Afinne_arabic.csv", 'r', encoding="utf-8")
scores = {} # initialize an empty dictionary
for line in afinnfile:
    line = line.strip()
    term = line.split(',')[0]
    score = line.split(',')[1]
    try:
        scores[term] = float(score) # Conver the score to an integer. It was parsed as a string.
    except:
        None
afinnfile.close()
print(len(scores))
# print(scores.items( )) # Print every (term, score) pair in the dictionary

42772


- make score for each tweets in sentiment file
- make newWords dectionary for words that not exist in clean_tweets file

In [6]:
tweets = open("/kaggle/working/clean_tweets.txt", 'r', encoding="utf-8")
sentiments = open("/kaggle/working/3_sentiment.txt", 'w', encoding="utf-8")
newWords = {}
for line in tweets:
    line = line.strip()
    total_score = 0
    for word in line.split(' '):
        wordScore = scores.get(word, 0)
        total_score = total_score + wordScore
        if wordScore == 0 and word != 'URL' and word != 'User':
            newWords[word] = newWords.get(word, 0)
    sentiments.write(str(round(total_score, 3)) + "\n")        
tweets.close()
sentiments.close()
print(len(newWords))

3573


give score for new words in newWords dictionary

In [7]:
tweets = open("/kaggle/working/clean_tweets.txt", 'r', encoding="utf-8")
sentiments = open("/kaggle/working/3_sentiment.txt", 'r', encoding="utf-8")
result = open("/kaggle/working/3_stdout.txt", 'w', encoding="utf-8")

tweets_scores = sentiments.read().split('\n')
word_score = None
for word in newWords.keys():
    word_score = 0
    index = 0
    for line in tweets:
        line = line.split(' ')
        if word in line:
            word_score += float(tweets_scores[index].strip())
        index += 1
    if word_score > 0:
        newWords[word] = 1
    elif word_score < 0:
        newWords[word] = -1
    else:
        newWords[word] = 0
    result.write(word +' '+ str(newWords[word]) +'\n')
    tweets.seek(0)
tweets.close()
print(newWords)

{'ميسي': -1, 'وينه': -1, 'العراق': -1, 'الصدر': -1, 'عشاق': -1, 'قاموا': -1, 'بإنشا': -1, 'يسمح': -1, 'لك': -1, 'بمعرفة': -1, 'المسافة': -1, 'والملاكم': -1, 'المكسيكي': -1, 'ساول': -1, 'الفاريز': -1, 'والموقع': -1, 'يحدد': -1, 'بأمان': -1, 'حاليا': -1, 'بينهما': -1, 'تشير': -1, 'إلى': -1, 'الأمان': -1, 'كأس_العالم_قطر_2022': -1, 'مونديال_قطر_2022': -1, 'الارجنتين': -1, 'Messi': -1, 'كأس_العالم': -1, 'السعوديه_المكسي': -1, 'يتقابون': -1, 'الممشى': -1, 'الليل': -1, 'حديثي': -1, 'لقناة': -1, 'الجزيرة': -1, 'كسرنا': -1, 'عينه': -1, 'واليوم': -1, 'إيران': -1, 'أمريكا': -1, '@User': -1, 'Qatar2022': -1, 'قطر_كأس_العالم_2022': -1, 'MAMA2022': -1, 'قالوا': -1, 'يهمنا': -1, 'فعلته': -1, 'حياتك': -1, 'حياتنا': -1, 'اخبرونا': -1, 'احمد_حسام_ميدو': -1, 'الشركة': -1, 'الراعية': -1, 'للنجم': -1, 'الأرجنتيني': -1, 'ولو': -1, 'الهدف': -1, 'لرونالدو': -1, 'سيتخطى': -1, 'الحين': -1, 'الطريقه': -1, 'أكيد': -1, 'عندنا': -1, 'المقاهي': -1, 'الأخضر': -1, 'وطبيعي': -1, 'يشوف': -1, 'على': -1, 'الجوال': -1, 'ن

## feature extraction
- N-grams are continuous sequences of words or symbols or tokens in a document.
- They come into play when we deal with text data in NLP(Natural Language Processing) tasks.

In [8]:
tweets = open("/kaggle/working/clean_tweets.txt", 'r', encoding="utf-8")
afinnfile = open("/kaggle/input/sentiment-arabic-tweets/3_Afinne_arabic.csv", 'r', encoding="utf-8")
f = open("/kaggle/working/featuresAndLabels.csv", 'w',newline='',encoding='utf-8-sig')
writer = csv.writer(f)

scores = {} # initialize an empty dictionary
for line in afinnfile:
    line = line.strip()
    term = line.split(',')[0]
    score = line.split(',')[1]
    try:
        scores[term] = float(score) # Conver the score to an integer. It was parsed as a string.
    except:
        None
afinnfile.close()

# Function to extract lexicon items
def extract(g, lex):
    if ' '.join(g) in lex.keys():
        return lex.get(' '.join(g))
    elif g[0] in lex.keys():
        #print(g[0])
        #print(lex.get(' '.join(g)))
        return lex.get(g[0])
    else:
        pass

# Your task
mylist = list()
mylist.append(['tweet', 'feature'])
for line in tweets:
    line = line.strip()
    out = [extract(g, scores) for g in ngrams(line.split(), 2) if extract(g, scores)]
    print(out)
    out = sum(out)
    if out > 0:
        mylist.append([line,"positive"])
    elif out < 0:
        mylist.append([line,"negative"])
    else:
        mylist.append([line,'neutral'])
writer.writerows(mylist)
tweets.close()
f.close()

[0.119, 0.772, 0.119]
[0.319, 0.015, -0.279, 0.1, -0.184, 0.104, -0.05]
[0.176, 0.1]
[]
[]
[-0.806, 0.119, -0.195, -0.9]
[0.511, -0.195, 0.276, 0.467, 1.243]
[-0.177, 1.028, 0.104, -0.208, 0.119, -0.208, 0.119, -0.546, -0.177]
[0.915, 0.331, 0.287, 0.751, 0.988, -0.043]
[]
[-1.615, 1.075, 0.349, 0.788, -0.016, -1.615, -0.05, 0.294, -0.344, 0.805, -1.615]
[0.176, -0.108, 0.104, 0.015, 0.176]
[0.023, 0.176, 0.119, -0.104, 2.005, 0.119, 0.043, 0.276, 0.615, -0.408, 0.176, -0.501, -0.806, 0.176, 0.559]
[1.028]
[-0.195, -0.177, -0.752, 0.754, -0.11, 0.278, 0.011, 0.757, 0.249, 0.119, 0.015, 0.176]
[0.1, -0.177, -0.181, 0.104, 0.477, 0.764, -0.546, 0.119]
[0.132, 0.119, 0.663, 0.772, -0.143, -0.433, 0.011, -0.61, 0.121, 0.155, 0.119, 0.79]
[-0.794, 0.181, 0.077, -0.794, -0.433]
[1.028]
[1.028]
[1.028]
[-0.11, 0.119, 0.554]
[-0.195, 0.059, -0.208, -0.709, 0.092]
[0.058, 0.119, -0.557, -0.07, 0.572, 0.351, 1.215]
[]
[-0.11, 0.748, 0.477, 0.142, 1.034, -0.461, 0.11, 0.834, 0.56, 0.283, -0.11, 0

In [9]:
df = pd.read_csv('/kaggle/working/featuresAndLabels.csv')
# df.head()
# df.info()
# df.isna().sum()
# df['feature'].value_counts()
x=df['tweet'].values
print(x.shape)
y=df['feature'].values
print(y.shape)

(469,)
(469,)
